# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="017_finetuned_embedding",
    RUN_DESCRIPTION="""
# Finetuned Embedding

Finetune the embedding model "Snowflake/snowflake-arctic-embed-m-v1.5" on 6k pairs of qa generated dataset (by GPT-4o-mini)

## Changelog
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=False,
    RECREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    DEBUG=False,
)

ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "017_finetuned_embedding",
  "RUN_DESCRIPTION": "\n# Finetuned Embedding\n\nFinetune the embedding model \"Snowflake/snowflake-arctic-embed-m-v1.5\" on 6k pairs of qa generated dataset (by GPT-4o-mini)\n\n## Changelog\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_INDEX": false,
  "RECREATE_RETRIEVAL_EVAL_DATASET": false,
  "RECREATE_RESPONSE_EVAL_DATASET": false
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-10 11:00:21.956 | INFO     | src.run.cfg:init:141 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix
2024-08-10 11:00:26.646 | INFO     | src.run.cfg:init:159 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 017_finetuned_embedding...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-08-10 11:00:26.876 | INFO     | src.run.cfg:init:168 - Notebook-generated artifacts are persisted at data/017_finetuned_embedding


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "017_finetuned_embedding",
    "RUN_DESCRIPTION": "\n# Finetuned Embedding\n\nFinetune the embedding model \"Snowflake/snowflake-arctic-embed-m-v1.5\" on 6k pairs of qa generated dataset (by GPT-4o-mini)\n\n## Changelog\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_INDEX": false,
    "RECREATE_RETRIEVAL_EVAL_DATASET": false,
    "RECREATE_RESPONSE_EVAL_DATASET": false
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/009_hybrid_retriever/storage_context",
  "db_collection": "review_rec_bot__009_hybrid_retriever__huggingface__Snowflake_snowflake_arctic_embed_m_v1_5",
  "notebook_cache_dp": "data/017_finetuned_embedding",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "togetherai",
    "llm_model_name": "meta-llama/Meta-Llama-3.1-8B-Instr

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-10 11:00:36.733 | INFO     | __main__:<module>:6 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
input_data = data
if ARGS.TESTING:
    input_data = data[:20]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-10 11:00:37.015 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [13]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [14]:
from llama_index.core import Document

documents = []
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in ('text')}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-08-10 11:00:39.348 | INFO     | __main__:<module>:20 - [COLLECT] len(documents)=5240


#### Check document embedding text

In [15]:
from llama_index.core.schema import MetadataMode

In [16]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [17]:
llm, embed_model = cfg.setup_llm()

WARNI [sentence_transformers.SentenceTransformer] You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [18]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "togetherai",
  "llm_model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
  "embedding_provider": "huggingface",
  "embedding_model_name": "./data/finetune_embedding/finetuned_model",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [19]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [20]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [21]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg, qdrantdb)

db_collection = qdrantdb.get_collection(cfg.db_collection)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    enable_hybrid=False,
    prefer_grpc=True
)

2024-08-10 11:00:48.335 | INFO     | src.run.orchestrator:setup_db:37 - Use existing Qdrant collection: review_rec_bot__009_hybrid_retriever__huggingface__Snowflake_snowflake_arctic_embed_m_v1_5
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


# Index Embeddings

In [22]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
# from llama_index.core.node_parser import SentenceSplitter

In [23]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [24]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-08-10 11:00:50.082 | INFO     | __main__:<module>:7 - Loading Storage Context from data/009_hybrid_retriever/storage_context...


In [25]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-10 11:00:58.008 | INFO     | __main__:<module>:1 - Indexing 5240 documents into VectorStoreIndex took 8s
2024-08-10 11:00:58.009 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=10219


# Analyze Chunks

In [26]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly':

# Construct Retriever

In [27]:
from llama_index.core.retrievers import VectorIndexRetriever

In [28]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 10,
  "retrieval_dense_top_k": 5,
  "retrieval_sparse_top_k": 15,
  "retrieval_similarity_cutoff": null,
  "rerank_top_k": 5,
  "rerank_model_name": "BAAI/bge-reranker-large"
}


In [29]:
# configure retriever
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=cfg.retrieval_cfg.retrieval_dense_top_k,
    # sparse_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
)

In [30]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** db6b2d69-c359-4927-8849-ff1581e2eada<br>**Similarity:** 3.1722183227539062<br>**Text:** A good grocery store, just requires a car to get to (or bus). This used to be somewhere they shuttled local university students from Penn to come get groceries; not sure if they still do that.

There's almost always a good deal to be had, fresh produce and meat, and feels like a truly suburban grocery store. Lines are long at times, but the only real problem is the distance from the city if you don't have a car.<br>**Metadata:** {'review_id': 'hKTTE9mOS1tXI6gDbDU_Mw', 'user_id': 'nnwBdqGHIAJQ5QX9lHOtrQ', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2009-01-20T16:11:51', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 8ec77c6a-6c2a-4b53-b928-259a4b5ccf8b<br>**Similarity:** 3.0762240886688232<br>**Text:** This grocery store SUCKS!  But it's the closest one to my house, so I go there for things that I need fast.  The produce is the worst possible.  I don't think the prices are very good, but then again it seems that all grocery stores in Philly are outrageously priced.  Employees are the slowest moving in the world!<br>**Metadata:** {'review_id': 'krw5FlAWXS16NkkqViK-wg', 'user_id': 'B-d6QdtkkoLHCyIyuT6y4A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2011-05-12T20:29:04', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 2020bdc4-4d5b-419d-9581-81cd7076823c<br>**Similarity:** 3.0317909717559814<br>**Text:** Free parking garage.  Eat in grocery store or deli type restaurant....everything is by the pound.<br>**Metadata:** {'review_id': 'vQxUaNanrYGZ_X3qCiIC3g', 'user_id': 'btAiOvMIfAbP5Sc5fAR8zA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-07-08T18:07:49', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

**Node ID:** ea7d973e-a656-4e3a-93c6-bd64ac553b3f<br>**Similarity:** 2.975971221923828<br>**Text:** Not the best grocery store in the world but the prices are reasonable. My cashier kept literally screaming for help.<br>**Metadata:** {'review_id': 'n8JpZwGLs3BwBzIPGlz7BQ', 'user_id': 'oD1WErpp65gbk29ErcDsLw', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2013-10-12T15:53:42', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

...

**Node ID:** 3ab49015-e92a-4891-8557-c9f42c886837<br>**Similarity:** 2.6478774547576904<br>**Text:** If you're looking for great tasting, high quality Italian food then this is the place for you. They make their own bread that is to die for. They have the best manicotti that I've ever had. Yum! There isn't a single bad thing on the menu. They also offer a unique Italian grocery selection with imported and domestic Italian products that are very hard to find at regular grocery stores. Support small business and give this place a try!<br>**Metadata:** {'review_id': '7jSEUBirxk_ghx8OI9myQg', 'user_id': 'DIXNEOmHO6D-zOM0X9fNEg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 2, 'funny': 0, 'cool': 1, 'date': '2014-01-03T22:13:26', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** b1c55b3a-ce09-46d4-ae03-795999f88bc8<br>**Similarity:** 2.636214256286621<br>**Text:** Really neat concept!  There is a small gourmet grocery. A soup and sandwich station, a salad station and a combination beer, wine and coffee bar.  Loved the frozen rose!<br>**Metadata:** {'review_id': 'b4aG7Vq86gHcAIt99FiJrA', 'user_id': 'HBjWfVa3BaRxFAH6TDc1Nw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-09-23T02:17:54', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

**Node ID:** e002734f-0e43-47e9-84a7-1b6a6910f4f6<br>**Similarity:** 2.5576841831207275<br>**Text:** A must stop in Nashville<br>**Metadata:** {'review_id': 'BbJRG7HkGx0S3SuvvpVvIQ', 'user_id': '26nVu2FZTrYt3PEMXLCR0A', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2013-09-28T04:48:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

**Node ID:** 139ef666-1ca5-4606-8799-25705d5f50aa<br>**Similarity:** 2.4747650623321533<br>**Text:** Wow!!!! Perfect and delicious. As good as the best delis in NYC or Boston. Fast service and nice folks. Plus the little grocery of hard to find Italian specialties !!! Fantastic !! Yum !! Come hungry !!<br>**Metadata:** {'review_id': 'Wxw8_PKy98_Fo3yLKcwTnA', 'user_id': 'HnnDiOQETwAEMfGZfLJ9gg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-06-20T16:36:12', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** 973a0309-7588-44d7-ae3d-0e8f8d5fc159<br>**Similarity:** 2.4747650623321533<br>**Text:** Wow!!!! Perfect and delicious. As good as the best delis in NYC or Boston. Fast service and nice folks. Plus the little grocery of hard to find Italian specialties !!! Fantastic !! Yum !! Come hungry !!<br>**Metadata:** {'review_id': 'wd5V8nsdvBkmi6ra-bh1hg', 'user_id': 'HnnDiOQETwAEMfGZfLJ9gg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 2, 'funny': 0, 'cool': 0, 'date': '2016-03-31T17:40:24', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

In [60]:
# test_query = nodes[3].text
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)

# TODO: Reconsider this test.
# Currently it aims to make sure the vector retriever, 
# when given a piece of text extracted from a node, 
# would return the original node.
# But what is the value of it?

# Check if the original node in top 10:
found = False
for vector_result in vector_results[:10]:
    target_id = bm25_results[0].id_
    if vector_result.id_ == target_id:
        found = True
        break
assert found, "Can not find the original node in the top 10 vector results"
# assert bm25_results[0].id_ == vector_results[0].id_, "Different vector_retriever nodes vs bm25_retriever nodes"

In [61]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [62]:
query = "What are some places to enjoy cold brew coffee"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.EMBED))
    print("\n")

Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee, and I was amazed by their unique flavors and brewing methods. The atmosphere was perfect for relaxing and enjoying a refreshing drink."


**Node ID:** cc79cdde-1351-4cc3-8613-08dbb5afbb92<br>**Similarity:** 0.06361048242394585<br>**Text:** I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.<br>

review_stars: 4
biz_name: Wawa
biz_address: 3604 Chestnut St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Restaurants', 'Automotive', 'Delis', 'Gas Stations', 'Food', 'Coffee & Tea', 'Sandwiches', 'Convenience Stores']

I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.




**Node ID:** 17d515f7-ee7b-4ad8-ba5e-7ade77e44922<br>**Similarity:** 0.03333333333333333<br>**Text:** This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee.<br>

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee.




...

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

Best neighbors ever!!

If I would wish for a dream neighbor it would be Caviar & Bananas. 
You know, that neighbor that is never in your business but always there when you need them! 
The one that always have a cup of sugar and 2 eggs. Well, this place is it, they fulfill so many of my needs!

Need a healthy but still delicious lunch? Check!
Need a bottle of wine? Check!
Need a cold brew coffee with just a splash of almond milk? Check!




**Node ID:** a3f8a081-214b-405a-9012-b4c28d7f9627<br>**Similarity:** 0.02964254577157803<br>**Text:** GREAT BREWS!!!
This place isn't always the first place folks think of when they decide to visit a brewery in Indy... but it should be!! It's on the outskirts of town, but well worth heading over to! They have some wonderful mixes/brews... and their many awards prove it! 

It has a nice sized tasting room that is comfy welcoming. Not to mention the ever changing chalk board with some awesome art work to fancy up their list of what is on tap. 

GREAT place and wonderful people!!!<br>

review_stars: 5
biz_name: Bier Brewery and Tap Room
biz_address: 5133 E 65th St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Food', 'Beer', 'Wine & Spirits', 'Breweries']

GREAT BREWS!!!
This place isn't always the first place folks think of when they decide to visit a brewery in Indy... but it should be!! It's on the outskirts of town, but well worth heading over to! They have some wonderful mixes/brews... and their many awards prove it! 

It has a nice sized tasting room that is comfy welcoming. Not to mention the ever changing chalk board with some awesome art work to fancy up their list of what is on tap. 

GREAT place and wonderful people!!!




**Node ID:** 36991e23-3f96-4bcd-ac95-70c84e20e5f0<br>**Similarity:** 0.02904040404040404<br>**Text:** If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.




## Retrieval Evaluation

### Synthetic

In [63]:
from src.run.eval import RetrievalEvaluator

In [64]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-10 11:09:33.527 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:79 - Loading retrieval_eval_nodes from data/015_revamp_synthetic_eval/retrieval_synthetic_eval_dataset.json...
2024-08-10 11:09:33.537 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:85 - Loading existing synthetic retrieval eval dataset at data/015_revamp_synthetic_eval/retrieval_synthetic_eval_dataset.json...


In [65]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'dbfb4988-5308-4097-8f58-b6606a3e845f': 'What are some good parking options available near LAPLCO Blvd & Manhattan?',
 '9d2fa648-f5a2-40e1-81c2-d5020cf0fb2c': 'What are some reputable shipping companies known for handling packages without damage?',
 'de83b07a-743a-44e3-b3d2-7286f3897345': 'What are some affordable parking options in Midtown Nashville?',
 '0dfb1efe-e023-42a4-9540-e1e05fed0fa7': 'What are some businesses or locations that have received poor reviews for customer service?',
 'edfd8ea6-5424-4c1c-b01a-6dc67ba9aca0': 'What are some ice cream shops that offer unique flavors and a nostalgic atmosphere?',
 '6d9dc768-30c5-46bf-8975-920250d5bebd': 'What are some popular ice cream shops in Nashville that offer waffle cones?',
 'ab8c899e-7c30-4b0a-b2da-d742415933c3': 'What are some sushi restaurants that offer a la carte options instead of tasting menus?',
 '0693c761-676f-45a7-963e-cfca4b8ce418': 'What are some cafes known for their breakfast sandwiches and high-quality coffee?',
 

In [66]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
parking options LAPLCO Blvd Manhattan
"I recently visited LAPLCO Blvd and found several convenient parking spots nearby. The parking garage on Manhattan was particularly spacious and affordable, making it easy to access the area."
Generated queries:
reputable shipping companies package handling damage-free
"I recently used SafeShip for my package delivery, and I was impressed by their careful handling. My items arrived in perfect condition, and their customer service was top-notch!"
Generated queries:
affordable parking options midtown nashville
"I recently visited Midtown Nashville and found several affordable parking options. The lot on 21st Avenue was a great deal, and there were also street parking spots available that were reasonably priced."
Generated queries:
poor customer service reviews businesses locations
Generated queries:
ice cream shops unique flavors nostalgic atmosphere
"I visited a charming ice cream shop that had an amazing selection of unique flavo

In [67]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_10_retrieval_eval,0.633333,0.336706,0.063333,0.633333,0.336706,0.089754


In [68]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some good parking options available n...,[e398cffc-eb02-42e3-923a-604ae06c4889],"[e398cffc-eb02-42e3-923a-604ae06c4889, e2a971c...",1.0,1.000000,0.1,1.0,1.000000,0.220092
1,What are some reputable shipping companies kno...,[03e033d4-2831-439e-8ea6-a4946a41916a],"[03e033d4-2831-439e-8ea6-a4946a41916a, 57ee580...",1.0,1.000000,0.1,1.0,1.000000,0.220092
2,What are some affordable parking options in Mi...,[7bb1304d-9638-4b37-89b3-b8c852ffeff2],"[7bb1304d-9638-4b37-89b3-b8c852ffeff2, 1adefae...",1.0,1.000000,0.1,1.0,1.000000,0.220092
3,What are some businesses or locations that hav...,[70f23ccc-9c0d-478c-a3fa-602529082d56],"[77c48c1b-cde7-40d9-8d72-985e3ee10cc5, 411b6f6...",0.0,0.000000,0.0,0.0,0.000000,0.000000
4,What are some ice cream shops that offer uniqu...,[f89a64e7-2b86-4333-89fa-11a18dc7d897],"[d15b604d-f289-442b-9619-09260e0c29a6, acb8d30...",0.0,0.000000,0.0,0.0,0.000000,0.000000
5,What are some popular ice cream shops in Nashv...,[fd39867d-dddb-4ae8-abd2-9e0422dce67a],"[4bbb3710-e7d0-4107-b22b-dde342c80478, e5b8adb...",1.0,0.333333,0.1,1.0,0.333333,0.110046
6,What are some sushi restaurants that offer a l...,[5c86dd25-9c62-4117-ad76-87823a901d02],"[72a55725-0f5d-42a8-854f-7bf3bd4675cc, 5c86dd2...",1.0,0.500000,0.1,1.0,0.500000,0.138862
7,What are some cafes known for their breakfast ...,[07042f7e-af55-4f78-9c1b-5259eaa0cd11],"[f3710ad8-06f6-4c34-8ee6-437ef55e4dbe, 17d515f...",1.0,0.142857,0.1,1.0,0.142857,0.073364
8,What are some financial institutions known for...,[2b20d945-b172-4a11-b82f-8c3238d42a16],"[a201a9b6-0205-40ea-994a-cc44498a4146, ffd2c58...",0.0,0.000000,0.0,0.0,0.000000,0.000000
9,What are some restaurants known for their reli...,[955626ac-20b8-422e-9562-302188e37c11],"[7b128f20-bb3f-485f-ab00-b2f77f394138, 7ba1022...",0.0,0.000000,0.0,0.0,0.000000,0.000000


#### Error Analysis

In [69]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
3,What are some businesses or locations that have received poor reviews for customer service?,[70f23ccc-9c0d-478c-a3fa-602529082d56],"[77c48c1b-cde7-40d9-8d72-985e3ee10cc5, 411b6f66-9951-4032-9c1b-6e9045069b6f, ff7657e4-8d48-4d33-9b42-099cd02f3a2c, d59bbaed-472a-42dc-8fdd-d38b5a9d0242, 928aa9e5-86f8-4315-9405-4c8038b64ea4, 55cee61e-a0fc-4e85-8ae1-b13861f30f71, ffd2c584-2512-4020-918c-95d1bc2922e8, 1621c153-c0ee-4528-b1d7-55e7a5db255f, 1d1c801f-bcf9-44c7-860b-299295930934, 84195783-42dd-44e3-b97b-6a75c95525f1]",0.0,0.0,0.0,0.0,0.0,0.0
4,What are some ice cream shops that offer unique flavors and a nostalgic atmosphere?,[f89a64e7-2b86-4333-89fa-11a18dc7d897],"[d15b604d-f289-442b-9619-09260e0c29a6, acb8d309-f3a0-40e8-b608-5e3e996f602a, 395772c0-a786-4791-9de0-711e9c22ff3b, a67f63c6-a802-413c-806a-16735b28f347, 49c3f474-ef09-4bf0-a248-8219c80e5bc0, e7c46277-9752-4130-8efe-535dd7554e44, 43a57307-7784-4df7-9100-c32f6ea01414, 330dfd87-96d9-4925-a906-4df81dbe542b, 3832bae5-b6d2-4bca-860a-d928fb323bb7, 103fc85f-6ad4-4bd3-935f-fd68fd810a50]",0.0,0.0,0.0,0.0,0.0,0.0
8,What are some financial institutions known for better customer service and support?,[2b20d945-b172-4a11-b82f-8c3238d42a16],"[a201a9b6-0205-40ea-994a-cc44498a4146, ffd2c584-2512-4020-918c-95d1bc2922e8, 2d38afc3-1b57-4108-9471-e3fdbab1fa25, ad52a0fb-9725-40f6-83f6-2c6097377d12, bfe518c9-84e1-4092-924c-8a2395af1cf0, 66063217-05c8-44f1-941c-3ee29e1d0dd7, eb74777c-c297-4d8b-b430-e97c2b4bf1e0, 4caedf20-1d96-431b-8fd6-b906e9b1bea7, 77c48c1b-cde7-40d9-8d72-985e3ee10cc5, bbf6cd65-ab29-4594-b2ef-ae60b9334126]",0.0,0.0,0.0,0.0,0.0,0.0
9,What are some restaurants known for their reliable service and consistent food quality?,[955626ac-20b8-422e-9562-302188e37c11],"[7b128f20-bb3f-485f-ab00-b2f77f394138, 7ba10220-0c7a-44f4-9e52-0202803df53e, d2fce3e0-4278-4435-87b5-57ef77e4b24a, 02cd763b-dd78-460e-83a0-d71960c11cfa, da7b4cf5-0648-4958-9397-0eda17f9c616, 18bd79e3-060d-4032-a6f6-e9f239dc33b2, 7afbd839-b294-4880-bad1-1d59abc8c4a5, 713a3fb7-a5df-41d3-abd8-4ac302e523a1, f6fce834-e8e4-4fd2-b92a-ba0bea296d4c, 713747c0-ed84-4dde-9163-f23f137fdc33]",0.0,0.0,0.0,0.0,0.0,0.0
13,What are some cafes or restaurants that offer healthy breakfast options?,[c92d191f-fdbb-4442-98cb-a823ead8df82],"[d6d04b18-dcd5-4a18-9973-8c6f4fe9e88c, 923a03c6-1a15-44a5-ba25-03917e37a4d7, 7c81a4b6-381e-45db-8bdb-bd48ffab4d48, 6a2dd440-38c1-4de2-93af-dc7cbfb8a164, 3f79a723-f810-4143-af50-82cd6d1317bc, bb6908e5-9a9b-4f11-8131-52507f41c86b, a8a4e7b0-11fc-4a3b-80a6-9791f90e452d, 32c72efa-4362-40f6-9653-2863bbdd03b5, 1aa39de5-e1e6-4a67-8be8-c390e7eb5d20, fa52e4c7-bbb8-4aa9-98ca-f0810817b45c]",0.0,0.0,0.0,0.0,0.0,0.0
17,What are some engaging activities or attractions for children that can create lasting memories?,[e3102655-7da6-4b01-aa15-ed64966ebeda],"[5ea6d748-6b62-4235-b28e-82c9f1e629c6, c87cef5c-e8b5-48cb-b1e4-cd1177b9adb9, dabb332f-7d10-42e9-a98b-406c14e26e36, 67e9bda6-f0f3-4d00-9b4e-9c90f4be0bad, b11a0607-dd73-41d7-8948-a72b7d2d160c, ee0e0359-0580-434e-8209-aac9f2964360, b39e2841-b715-4cfd-ad2d-528ab1b8a186, 7c3dd316-db8b-4fa9-856b-97328574fad6, 9d7da9f4-2b1f-4456-88be-c0a0066d46ed, 7c914e5b-59a7-47f8-87d3-ac6bb8a56f93]",0.0,0.0,0.0,0.0,0.0,0.0
21,What are some restaurants known for their excellent customer service and knowledgeable staff?,[25e9f69b-faa6-43a3-af08-eb67e8f28d1d],"[8ae69a21-f331-4d77-afe5-daa0de4bc506, eb87b91d-7625-4966-862c-685fc376e775, 0a364b3b-525a-41cc-8e6c-964e8b1512f7, 922ff873-709b-4488-9288-c0419c650bba, c11a1dd0-0f0b-4e36-a460-fbdf55e45b25, a98648fc-94ea-488c-ab6f-9041d5fdf0ab, 3bc238bb-c1dc-4b60-9b96-261751209886, cdba436b-c6e9-4785-bfed-cf6d756d2779, 4ed75cca-c313-4b75-8f21-e65fbb2ebcfc, 3377961f-0b68-44c9-ac77-d8941156e582]",0.0,0.0,0.0,0.0,0.0,0.0
22,What are some popular tasting rooms or wine bars to visit in the Funk Zone?,[030e5249-5dac-4527-9d7b-599b30087686],"[b67207c0-84fe-45d7-8691-5812afdc6dea, d7a341fe-07a2-4181-a

In [70]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some businesses or locations that have received poor reviews for customer service?

- Expected contexts:

review_stars: 1
biz_name: H&M
biz_address: 827-833 State St
biz_city: Santa Barbara
biz_state: CA
biz_categories: ["Women's Clothing", 'Accessories', "Children's Clothing", "Men's Clothing", 'Adult', 'Shopping', 'Fashion']

Awful.

- Retrieved contexts:

+ Context #1:

review_stars: 1
biz_name: Chase JP Morgan Bank Credit Card Services
biz_address: 
biz_city: Wilmington
biz_state: DE
biz_categories: ['Banks & Credit Unions', 'Financial Services']

Every person I have spoken with has a different way to resolve the matter, yet nothing is resolved.  This has turned into quite the night mare for me.  Very very dissatisfied with the customer service at Chase Credit Card

+ Context #2:

review_stars: 1
biz_name: Famous Footwear
biz_address: 8522 Eager Road, Dierbergs Brentwood Point
biz_city: Brentwood
biz_state: MO
biz_categories: ['

In [71]:
query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.EMBED))
    print("\n")

Generated queries:
grocery stores nashville curated items local products
"I recently visited a grocery store in Nashville that had an amazing selection of curated items and unique local products. The atmosphere was great, and I found some fantastic local jams and artisanal cheeses!"


**Node ID:** 7bb1304d-9638-4b37-89b3-b8c852ffeff2<br>**Similarity:** 0.04825870646766169<br>**Text:** Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.




**Node ID:** c11a1dd0-0f0b-4e36-a460-fbdf55e45b25<br>**Similarity:** 0.03333333333333333<br>**Text:** Stumbled upon this hidden gem of a store when we went to have dinner at of of the local restaurants.. 

They have some very unique items.. High quality which mean you get what you pay for! They also have several local artist/products.. 

Customer service hands down excellent!<br>

review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

Stumbled upon this hidden gem of a store when we went to have dinner at of of the local restaurants.. 

They have some very unique items.. High quality which mean you get what you pay for! They also have several local artist/products.. 

Customer service hands down excellent!




...

review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

Hi, I'd like literally everything in this store. 

Seriously, if I had some serious cash in the bank, I'd purchase nearly every item in this store...if the size would fit my small body type. 

Honestly, this place is exactly what I'd like my apartment to look like. Woodsy, rustic, with a touch of urban. It appeals to the eyes, but even more the other senses, especially smell. The smells inside the store and the other products are my type of body products, spot on.

The biggest concern about the store is the price tag on most items. But, look at the quality and where it comes from and you can reason the price tag for a nice "treat yourself" moment every now and then. 

I particularly loved looking through the tons and tons of items throughout the store - body products, cologne, hair and beard grooming, clothes, shoes, bike

**Node ID:** 268c464d-a7ab-4440-8be1-2b0d5f731e64<br>**Similarity:** 0.029957522915269395<br>**Text:** At first, I defended this store because it's the only affordable grocery option for anyone south of South/north of Broad. But after several visits, I've realized it's okay for picking up a few items, but not a replacement for my regular stores (Shop Rite in South Philly/Jersey or Wegmans in Jersey). Yes, I still make regular grocery runs to Jersey, because the quality of the stores is so much better. I digress.<br>

review_stars: 2
biz_name: Pathmark
biz_address: 3021 Grays Ferry Ave
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Food', 'Grocery']

At first, I defended this store because it's the only affordable grocery option for anyone south of South/north of Broad. But after several visits, I've realized it's okay for picking up a few items, but not a replacement for my regular stores (Shop Rite in South Philly/Jersey or Wegmans in Jersey). Yes, I still make regular grocery runs to Jersey, because the quality of the stores is so much better. I digress.




**Node ID:** e25e5c68-a06b-46e7-bb4d-6275652fb170<br>**Similarity:** 0.029857397504456328<br>**Text:** They have an amazing selection of clothing, shoes, and all natural + local products. A little over priced ($320 for a pair of shoes!), but I guess if you have the money it'd be worth the visit. We bought some of their Bourbon Barrel Coffee which was SO not good and very expensive ($17/half pound). I like the concept of the store, but the prices of their products should be adjusted.<br>

review_stars: 4
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

They have an amazing selection of clothing, shoes, and all natural + local products. A little over priced ($320 for a pair of shoes!), but I guess if you have the money it'd be worth the visit. We bought some of their Bourbon Barrel Coffee which was SO not good and very expensive ($17/half pound). I like the concept of the store, but the prices of their products should be adjusted.




# Response

In [72]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine

In [73]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer()
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [74]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [75]:
# question = "where to find good cold brew coffee?"
question = "What are some gastropubs in Indianapolis that are known for their cocktails?"
response = query_engine.query(question)
display_response(response, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
gastropubs cocktails indianapolis
"I visited a fantastic gastropub in Indianapolis that had an impressive cocktail menu. The mixologist crafted unique drinks that perfectly complemented the delicious food."


**`Final Response:`** Based on the reviews, it seems that Village Tap Room is a great option for gastropubs in Indianapolis that serve excellent cocktails. They have a nice selection of hand-crafted cocktails, and reviewers have praised their craft cocktail offerings.


#### Referenced Paragraphs
**Village Tap Room**

> This is NOT Chumleys. Excellent local craft beer selection.  Hand crafted cocktails. Great food. Friendly staff. Check this place out!! (Review: `6LsSfJ5yH2m05agEKfgPcw`)

> Much improved over Chumley's.  Nice selection of local craft beer.  Craft cocktails.  Food was very good.  Great staff.   (Review: `48NsBnBkWSRWtEtQKuPCGw`)

> I ordered the steak burger medium with chipotle salsa (I was experimenting). The steak burger was well done as was my friend who also ordered medium. The buns were the cheap, wonderbread, kind you get in the 12-pack for a block party bbq. The chipotle salsa appeared to just be some salsa from a common brand container. This all cost me $9, which was ridiculous. 

The restaurant was pretty much empty, but I'm not that surprised. Come here for the drink special before it goes out of business! (Review: `J3DrcnIoOSppEF3rwboxag`)

**Bier Brewery and Tap Room**

> Their beer is some of the best in the area and the environment they have screams of coziness. One of the most underrated breweries in Indianapolis, Bier provides one of the finer experiences in breweries, so please check out this homely feeling place soon. Shout out to the hop gummy series! (Review: `B_vTDLOhkKfL3ALQsGrMlg`)

**Twin Peaks**

> McKenzie is very attentive..wont leave your beer empty not to mention her nice personality..beers are the coldest in Indianapolis..never disappointed here.. (Review: `OQLBodcmQonswWDXwm8IjQ`)

---

**`Source Node 1/5`**

**Node ID:** e120569d-8892-4519-9492-602dc11d0d68<br>**Similarity:** 1.355761170387268<br>**Text:** This is NOT Chumleys. Excellent local craft beer selection.  Hand crafted cocktails. Great food. ...<br>**Metadata:** {'review_id': '6LsSfJ5yH2m05agEKfgPcw', 'user_id': 'E1M3jc7hoYq9LWffruavFg', 'business_id': 'L_TT0BFmFwORAMaA83K54A', 'review_stars': 5, 'useful': 2, 'funny': 0, 'cool': 1, 'date': '2015-12-12T02:40:41', 'biz_name': 'Village Tap Room', 'biz_address': '838 Broad Ripple Ave', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8699114, 'biz_longitude': -86.1435773, 'biz_stars': 2.5, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': None, 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': "{'jukebox': True}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'False', 'Smoking': "u'no'", 'WheelchairAccessible': 'False', 'WiFi': "u'no'"}, 'biz_categories': ['Gastropubs', 'Cocktail Bars', 'Beer Bar', 'Bars', 'Restaurants', 'Nightlife'], 'biz_hours': None}<br>

...

**Node ID:** d8697745-bd4e-4a32-a29a-a9c3c5698e05<br>**Similarity:** -4.539366245269775<br>**Text:** Their beer is some of the best in the area and the environment they have screams of coziness. One...<br>**Metadata:** {'review_id': 'B_vTDLOhkKfL3ALQsGrMlg', 'user_id': '0ubLKzwF7F4mWzWLLZLGJA', 'business_id': '7clCBgNbd-x2Wj96lZ6Mjw', 'review_stars': 4, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2020-12-16T18:01:54', 'biz_name': 'Bier Brewery and Tap Room', 'biz_address': '5133 E 65th St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8755319, 'biz_longitude': -86.0828565, 'biz_stars': 4.5, 'biz_review_count': 139, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'beer_and_wine'", 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'False', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Beer', 'Wine & Spirits', 'Breweries'], 'biz_hours': {'Friday': '13:0-21:0', 'Monday': None, 'Saturday': '12:0-21:0', 'Sunday': '12:0-19:0', 'Thursday': '15:0-21:0', 'Tuesday': '15:0-21:0', 'Wednesday': '15:0-21:0'}}<br>

---

**`Source Node 5/5`**

**Node ID:** 4218decc-0031-4d7c-964a-14dfa7abb7ee<br>**Similarity:** -5.775516510009766<br>**Text:** McKenzie is very attentive..wont leave your beer empty not to mention her nice personality..beers...<br>**Metadata:** {'review_id': 'OQLBodcmQonswWDXwm8IjQ', 'user_id': 'BrCzG5CfPoQCrVgeuB0Gqw', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-10-29T18:49:08', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'BYOB': None, 'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False}", 'NoiseLevel': "'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Sports Bars', 'American (New)', 'American (Traditional)', 'Nightlife', 'Bars', 'Restaurants'], 'biz_hours': {'Friday': '11:0-19:0', 'Monday': '0:0-0:0', 'Saturday': '11:0-1:0', 'Sunday': '11:0-0:0', 'Thursday': '11:0-22:0', 'Tuesday': '11:0-0:0', 'Wednesday': '11:0-0:0'}}<br>

{'e120569d-8892-4519-9492-602dc11d0d68': {'review_id': '6LsSfJ5yH2m05agEKfgPcw',
  'user_id': 'E1M3jc7hoYq9LWffruavFg',
  'business_id': 'L_TT0BFmFwORAMaA83K54A',
  'review_stars': 5,
  'useful': 2,
  'funny': 0,
  'cool': 1,
  'date': '2015-12-12T02:40:41',
  'biz_name': 'Village Tap Room',
  'biz_address': '838 Broad Ripple Ave',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46220',
  'biz_latitude': 39.8699114,
  'biz_longitude': -86.1435773,
  'biz_stars': 2.5,
  'biz_review_count': 23,
  'biz_is_open': 0,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "u'none'",
   'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}",
   'BYOB': None,
   'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}",
   'BikeParking': 'False',
   'BusinessAccept

## Response Evaluation

In [76]:
from src.run.eval import ResponseEvaluator

In [77]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [78]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-10 11:13:41.836 | INFO     | src.run.eval.response:generate_synthetic_dataset:39 - Sampling 30 documents for response evaluation...
2024-08-10 11:13:41.843 | INFO     | src.run.eval.response:generate_synthetic_dataset:95 - Loading existing synthetic response eval dataset at data/015_revamp_synthetic_eval/response_synthetic_eval_dataset.json...


In [79]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                        | 0/16 [00:00<?, ?it/s]

Generated queries:
sports bars dining experience indianapolis
"I visited a fantastic sports bar in Indianapolis called The District Tap. Not only do they have a great selection of beers, but their food menu is impressive, featuring delicious burgers and wings that elevate the dining experience."
Generated queries:
sports bars indianapolis good atmosphere watching games
"I visited a great sports bar in Indianapolis called The District Tap. The atmosphere was lively, perfect for watching games with friends, and they had a wide selection of beers on tap."
Generated queries:
wine bars new orleans great atmosphere
"I visited a fantastic wine bar in New Orleans that had an amazing atmosphere, perfect for a night out. The selection of wines was impressive, and the ambiance really made the experience special."
Generated queries:
ice cream shops unique cone options cookie cones
"I recently visited a fantastic ice cream shop that had the most amazing variety of cone options, including cookie con

Batch processing of predictions:   0%|                                                                                                                                                                        | 0/14 [00:00<?, ?it/s]

Generated queries:
ice cream shops unique flavors toppings nashville
"I visited a fantastic ice cream shop in Nashville that had the most unique flavors, like lavender honey and spicy chocolate. They also offered a variety of interesting toppings that made my dessert unforgettable!"
Generated queries:
bakeries santa barbara delicious pastries community seating
"I visited a lovely bakery in Santa Barbara that not only serves the most delicious pastries but also has a cozy seating area perfect for enjoying a treat with friends. The atmosphere is very community-oriented, making it a great spot to relax."
Generated queries:
sports bars lively atmosphere indianapolis
"I visited a great sports bar in Indianapolis that had an electric vibe, perfect for watching games with friends. The atmosphere was lively, and they had plenty of screens to catch all the action."
Generated queries:
cafe unique coffee food options nashville
"I visited a charming cafe in Nashville that had an incredible selecti

Batch processing of predictions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:24<00:00,  1.74s/it]


In [80]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:12<00:00,  2.40s/it]


In [81]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,2.966667
mean_relevancy_score,0.833333
mean_faithfulness_score,0.900000


In [82]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are some trendy Japanese restaurants in N...,\nIf you're looking for trendy Japanese restau...,1.0,3.0,1.0,[One of my favorite restaurants in Nashville. ...
1,- What are some Italian restaurants in Reno th...,\nIt's often the case that family-owned restau...,0.0,2.0,0.0,[Chicken parm was good but nothing really spec...
2,What are some highly-rated wine bars in New Or...,\nIf you're looking for a unique wine-drinking...,1.0,2.0,1.0,[Wow!!! One of my top three new favorite place...
3,What are some popular wine bars in New Orleans...,\nIf you're looking for a unique wine-drinking...,1.0,2.0,1.0,[Nice variety of wines at very reasonable pric...
4,What are some ice cream shops that offer uniqu...,\nSome ice cream shops offer unique cone optio...,1.0,3.0,1.0,[While walking around downtown I kept seeing p...
5,- What are some coffeehouses in Edmonton that ...,\nThere are several coffeehouses in Edmonton t...,1.0,3.0,1.0,[I went there yesterday afternoon for the firs...
6,What are some highly-rated sports bars that of...,\nIf you're looking for a sports bar experienc...,1.0,2.0,1.0,[Some menu items such as the bowls are fairly ...
7,- What are some cocktail bars in Philadelphia ...,\nSome popular cocktail bars in Philadelphia t...,1.0,3.0,1.0,[The service is incredible and the bar is bust...
8,What are some sports bars in Indianapolis that...,"\nBased on the reviews, it seems that Twin Pea...",1.0,4.0,1.0,[Delicious food. The waitresses were so nice a...
9,What are some popular wine bars in New Orleans...,\nIf you're looking for a unique and enjoyable...,0.0,2.0,1.0,[Cool wine bar located in the CBD. They have a...


#### Error Analysis

In [83]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [84]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
1,- What are some Italian restaurants in Reno th...,\nIt's often the case that family-owned restau...,0.0,2.0,0.0,[Chicken parm was good but nothing really spec...
9,What are some popular wine bars in New Orleans...,\nIf you're looking for a unique and enjoyable...,0.0,2.0,1.0,[Cool wine bar located in the CBD. They have a...
25,- What are some mobile phone stores known for ...,"\nBased on the information provided, it seems ...",0.0,2.0,1.0,"[Okay, so after a couple of trips to this plac..."
20,- What are some sushi bars in Philadelphia tha...,\nIt seems that some sushi bars in Philadelphi...,0.0,3.0,0.0,"[Anyways. Food is great, management sucks. The..."
14,- What are some sports bars in Indianapolis th...,\nIt seems that Twin Peaks in Indianapolis has...,0.0,3.0,1.0,[I'm a big sports bar guy and a sports bar foo...
2,What are some highly-rated wine bars in New Or...,\nIf you're looking for a unique wine-drinking...,1.0,2.0,1.0,[Wow!!! One of my top three new favorite place...
3,What are some popular wine bars in New Orleans...,\nIf you're looking for a unique wine-drinking...,1.0,2.0,1.0,[Nice variety of wines at very reasonable pric...
6,What are some highly-rated sports bars that of...,\nIf you're looking for a sports bar experienc...,1.0,2.0,1.0,[Some menu items such as the bowls are fairly ...
18,- What are some sports bars in Indianapolis th...,\nSports bars that offer a lively atmosphere o...,1.0,2.0,1.0,[Delicious food. The waitresses were so nice a...
21,- What are some popular brunch spots in New Or...,\nIf you're looking for a great brunch spot in...,1.0,2.0,1.0,[This is my new brunch spot! Came here with a ...


In [85]:
response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
- What are some Italian restaurants in Reno that have a reputation for good customer service?

Context:
Chicken parm was good but nothing really special. Too many other good Italian restaurants in Reno and probably would not eat here again.

I'm still hunting for a decent Italian restaurant in Reno.  This isn't it.  

Macaroni Grill lost our business because of the poor customer service from the manager on staff, which is sad because this restaurant was our go to for our date night.  Got an order to go and when I got home the alfredo sauce was completely dried up.  I called and asked to speak to the manager and when I spoke to her and told her how our order was she said well I spoke to the server and all I can do is give you a discount next time you are here because she said your order was good when you picked it up.  She told me don't you know that the sauce soaks in to the noodles, which for a to go order they should add extra then for

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [86]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-08-10 11:29:19.489 | INFO     | src.run.eval.response:generate_curated_dataset:124 - Persisting curated response eval dataset at data/017_finetuned_embedding/response_curated_eval_dataset.json...


In [87]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
recommended restaurants Santa Barbara
"I recently visited Santa Barbara and dined at a fantastic restaurant called The Lark. The ambiance was great, and the food was delicious, especially the seasonal dishes. I highly recommend it!"


Batch processing of predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.20s/it]


In [88]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.42s/it]


In [89]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,3.0
mean_relevancy_score,1.0
mean_faithfulness_score,1.0


In [90]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's likely that Helena Avenue Bakery is a p...,1.0,3.0,1.0,[Best meal we had in SB. Most places were a li...


#### Answers for target questions

In [91]:
from IPython.display import Markdown

In [92]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [101]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are the recommended restaurants nearby Santa Barbara?


Generated queries:
recommended restaurants Santa Barbara nearby
"I recently dined at a fantastic place called The Lark in Santa Barbara. The ambiance was great, and they had an amazing selection of local wines and farm-to-table dishes. Definitely a must-try!"


**`Final Response:`** The Lark is a recommended restaurant in the area, located in a cool arts complex with a few other restaurants and bars.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Finally some good food in downtown Santa Barbara.  Everything on the menu is great and could not recommend more.  (Review: `8qeD6EhCxUEBj1dO4J4KfA`)

> In Santa Barbara? This was the place to chill and relax for some refreshments and snacks.  (Review: `-eNFOKe_VE4BCb6DYWabzg`)

> If you're a tourist and spending the weekend in Santa Barbara, I'd recommend to give this place a go for sure. If you're a local, then not so much.  (Review: `Mqh2anPpP_vvLtqCAk_1Xw`)

> Helena Avenue Bakery is a must-visit for those looking for a great breakfast place in Santa Barbara. Not only is it in Downtown Santa Barbara, but it is located in the trendy FUNK ZONE close to the wharf.  (Review: `MvkzsY6BH6Fb5AABgLyAhg`)

> This bakery is in a cool arts complex area with a few other restaurants and bars (we came to this same area for dinner the night before at The Lark). Very cool Santa Barbara feel.  (Review: `lkKYjYq47PlA_IMF4KtC1w`)

### Error Analysis

In [94]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [95]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's likely that Helena Avenue Bakery is a p...,1.0,3.0,1.0,[Best meal we had in SB. Most places were a li...


In [96]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [97]:
from src.run.utils import parse_collect_log, flatten_dict

In [98]:
collect_info = parse_collect_log(collect_fp)

In [99]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

2024-08-10 11:29:28.697 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-08-10 11:29:28.730 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-08-10 11:29:28.779 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-08-10 11:29:28.976 | INFO     | __main

# Clean up

In [100]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Archive